**ogs-jupyter-lab: DECOVALEX TH process: FEBEX type repository**

<!--- ![tu-dresden-blue.png](attachment:tu-dresden-blue.png) --->
<img src="ogs-jupyter-lab01.png" alt="drawing" width="300"/>

**Description**

DECOVALEX is an international benchmarking project for the DEvelopment of COupled models and their VALidation against Experiments in geosystems with an emphasis on deep geological repositories (https://decovalex.org/). The present test case comes from Task D (BMT3: Long term changes in EDZ due to THM and THC processes) of the 4th phase DECOVALEX-THMC (2004-2007) (https://decovalex.org/D-THMC/index.html).


**Input data:** Mesh

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import matplotlib.tri as tri

#file
file_name = "th_decovalex.vtu"
reader = vtk.vtkXMLUnstructuredGridReader()
reader.SetFileName(file_name)
reader.Update()  # Needed because of GetScalarRange
data = reader.GetOutput()
temperature = data.GetPointData().GetArray("T_ref")
pressure = data.GetPointData().GetArray("p_ref")
#points
points = data.GetPoints()
npts = points.GetNumberOfPoints()
x = vtk_to_numpy(points.GetData())
triang = tri.Triangulation(x[:,0], x[:,1])
#cells
triangles=  vtk_to_numpy(data.GetCells().GetData())
ntri = triangles.size//4  # number of cells
tri = np.take(triangles,[n for n in range(triangles.size) if n%4 != 0]).reshape(ntri,3)
#plots
fig, ax = plt.subplots(ncols=3, figsize=(14,8))
#plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.6)
plt.subplots_adjust(wspace=0.5)
#mesh
ax[0].triplot(x[:,0], x[:,1], tri)
ax[1].tricontour(x[:,0], x[:,1], tri, pressure, 16);
contour_right = ax[2].tricontour(triang, pressure)
fig.colorbar(contour_right,ax=ax[1],label='$p$ / [Pa]')
ax[2].tricontourf(x[:,0], x[:,1], tri, temperature, 12);
contour_right = ax[2].tricontourf(triang, temperature)
fig.colorbar(contour_right,ax=ax[2],label='$T$ / [K]')
plt.savefig("figures.png")

AttributeError: 'NoneType' object has no attribute 'GetNumberOfPoints'

**Running ogs using [ogs6py](https://github.com/joergbuchwald/ogs6py)**

In [ ]:
from ogs6py import ogs
#run ogs
PATH_OGS="/home/ok/ogs/build/release/bin/"
print("===============")
print(">>> run ogs <<<")
model = ogs.OGS(PROJECT_FILE="th_decovalex.prj")
model.geo.add_geom(filename="boundary.gml")
model.mesh.add_mesh(filename="th_decovalex.vtu")
model.run_model(path=PATH_OGS,LOGFILE="console.log")

**Extracting results using [VTUInterface](https://github.com/joergbuchwald/VTUinterface)**

In [ ]:
# read and process (point interpolation) vtu- and pvd-files 
import vtuIO
import matplotlib.tri as tri
print("=====================")
print(">>> print results <<<")
pvdfile=vtuIO.PVDIO("th_decovalex.pvd", dim=2)
xaxis =  [(i,0,0) for i in np.linspace(start=0.0, stop=17.5, num=100)]
r_x = np.array(xaxis)[:,0]
time = [0.06,0.7,8.0,18.0,28.0,38.0,388.0,1000.0]
for t in time:
    function_xaxis_t = pvdfile.read_point_set_data(t, 'T', pointsetarray=xaxis)
    plt.plot(r_x, function_xaxis_t, label='t='+str(t))
# plot formatting
titlestring="TH process: FEBEX type repository"
plt.title(titlestring)
#plt.xscale('log')
plt.xlabel('x')
plt.ylabel('Temperature')
plt.legend()
plt.grid()
plt.savefig("profile.png")
plt.show()

**Vertical cross-sections**

In [ ]:
m_plot=vtuIO.VTUIO("th_decovalex_ts_30_t_8.000000.vtu", dim=2)
triang=tri.Triangulation(m_plot.points[:,0],m_plot.points[:,1])
u_plot = m_plot.get_point_field("T")
p_plot = m_plot.get_point_field("p")
fig, ax = plt.subplots(ncols=2, figsize=(10,10))
contour_left = ax[0].tricontourf(triang, u_plot)
contour_right = ax[1].tricontourf(triang, p_plot)
fig.colorbar(contour_left,ax=ax[0],label='$T$ / [K]')
fig.colorbar(contour_right,ax=ax[1],label='$p$ / [MPa]')
plt.show()

In [ ]:
import time
print(time.ctime())

**Results have been shown using [matplotlib](https://matplotlib.org/)**

**OGS links**
- description: https://www.opengeosys.org/docs/benchmarks/hydro-thermal/decovalex-th/
- project file: https://gitlab.opengeosys.org/ogs/ogs/-/blob/master/Tests/Data/Parabolic/HT/StaggeredCoupling/ADecovalexTHMCBasedHTExample/th_decovalex.prj

**Credits**
- Jörg Buchwald for ogs6py and VTUInterface
- Wenqing and Jörg for this benchmark set up